In [1]:
data_var = ''

In [2]:
# Parameters
data_var = "2024-08-24"


In [3]:
import pandas as pd; pd.set_option('display.max_columns', None)
import os
from pycaret.classification import *
from scripts.PreparaDatasetV2 import *
from scripts.TeoriaRetornos import *

### Funções

In [4]:
def atualiza_dia(_jogos):
    _jogos = prepara_df(_jogos)

    _jogos = _jogos.reset_index(drop=True)

    return _jogos

### Pegando os jogos do dia

In [5]:
df_today = pd.read_csv(f'jogos_do_dia/{data_var}.csv')

### Atualizando os datasets que tem jogo hoje

In [6]:
temporada = pd.read_csv('data/Entire_Season.csv')
temporada = temporada[temporada['Date'] < data_var]
temporada = pd.concat([temporada, df_today])
ligas = df_today['League'].unique().tolist()
temporada = temporada[temporada['League'].isin(ligas)]
temporada.reset_index(drop=True, inplace=True)

### Cria variáveis na temporada e nos jogos do dia

In [7]:
games = atualiza_dia(temporada)
temporada = games[games['Date'] < data_var]
os.makedirs(f'apostas_do_dia/{data_var}', exist_ok=True)

In [8]:
jogos = games[games['Date'] == data_var]

jogos = jogos.drop(columns=['Home_Pts', 'Away_Pts',
       'Back_Home', 'PL_Home', 'Back_Away',
       'PL_Away', 'Back_Over', 'PL_Over', 'PL_Under', 'Back_HA_H', 'PL_HA_H',
       'PL_HA_A', ])

jogos

,Date,League,Time,Home,Away,Odds_H,Odds_A,Over_Line,Odds_Over,Odds_Under,HA_Line,HA_Odds_H,HA_Odds_A,Fixture ID,P(H),P(A),P(O),P(U),P_Diff,Avg_CG_H,DP_CG_H,CV_CG_H,Avg_Pontos_H,DP_Pontos_H,CV_Pontos_H,CG_H_Last_Game,Media_Odds_Geral_H,DP_Odds_Geral_H,CV_Odds_Geral_H,Saldo_Gols_H,Avg_CG_A,DP_CG_A,CV_CG_A,Avg_Pontos_A,DP_Pontos_A,CV_Pontos_A,CG_A_Last_Game,Media_Odds_Geral_A,DP_Odds_Geral_A,CV_Odds_Geral_A,Saldo_Gols_A,Goals_Last_H,Goals_Last_A,Last_Odd_H,Last_Odd_A,MediaCustoGolHome,MediaCustoGolAway,CV_ML,CV_Over,CV_HA,Retornos_BH_Acu,Avg_Retornos_BH,Custo_Retorno_BH,Avg_Porc_BH_Bookie,Avg_Porc_BH_Real,Dist_Porc_BH,Retornos_BA_Acu,Avg_Retornos_BA,Custo_Retorno_BA,Avg_Porc_BA_Bookie,Avg_Porc_BA_Real,Dist_Porc_BA
389,2024-08-24,Paraguai Lnb,21:00,Olimpia,Colonias Gold,1.36,2.81,142.5,1.84,1.82,0.0,0.00,0.00,W0MvgG1K,0.735294,0.355872,0.543478,0.549451,0.091166,138.966,25.123899,0.180792,1.8,1.643168,0.912871,131.97,2.014,0.485469,0.241047,-4.0,189.606,64.688676,0.341174,1.8,1.643168,0.912871,105.78,2.572,0.972584,0.378143,18.0,83,86,1.59,1.23,0.000,165.002,0.491753,0.007728,NaN,0.00,0.000,inf,0.000000,0.0,0.000000,0.04,0.008,226.250000,0.000000,0.0,0.000000
390,2024-08-24,Paraguai Lnb,22:15,Dep. San Jose,Felix Perez Cardozo,1.01,9.35,145.5,1.84,1.82,0.0,0.00,0.00,4fUhVC9D,0.990099,0.106952,0.543478,0.549451,0.097051,109.334,12.934681,0.118304,2.4,1.341641,0.559017,95.68,1.430,0.238747,0.166956,48.0,191.350,58.666216,0.306591,0.6,1.341641,2.236068,163.40,2.676,0.818126,0.305727,-38.0,92,76,1.04,2.15,107.910,353.336,1.138469,0.007728,NaN,0.53,0.106,0.094340,0.753623,0.8,0.046377,-3.58,-0.716,-11.662011,0.000000,0.0,0.000000
391,2024-08-24,Tailândia Tbl,02:00,Dunkin Raptors,Pathum Thani,1.16,4.71,166.5,1.83,1.83,0.0,0.00,0.00,YXCdNn3e,0.862069,0.212314,0.546448,0.546448,0.074383,0.000,0.000000,0.000000,0.0,0.000000,0.000000,0.00,0.000,0.000000,0.000000,0.0,0.000,0.000000,0.000000,0.0,0.000000,0.000000,0.00,0.000,0.000000,0.000000,0.0,76,60,1.34,3.09,0.000,0.000,0.855274,0.000000,NaN,0.00,0.000,inf,0.000000,0.0,0.000000,0.00,0.000,inf,0.000000,0.0,0.000000
392,2024-08-24,Tailândia Tbl,04:00,Chaophraya Thunder,Bangkok Tiger Thunder,2.55,1.49,166.5,1.83,1.83,-3.5,1.83,1.71,EsL4LQX7,0.392157,0.671141,0.546448,0.546448,0.063298,0.000,0.000000,0.000000,0.0,0.000000,0.000000,0.00,0.000,0.000000,0.000000,0.0,0.000,0.000000,0.000000,0.0,0.000000,0.000000,0.00,0.000,0.000000,0.000000,0.0,110,88,1.28,1.69,0.000,0.000,0.371056,0.000000,0.047939,0.00,0.000,inf,0.000000,0.0,0.000000,0.00,0.000,inf,0.000000,0.0,0.000000
393,2024-08-24,Europa Copa Europeia 3X3 Feminina,14:50,Países Baixos 3x3 F,Alemanha 3x3 F,1.35,2.83,31.5,1.84,1.82,0.0,0.00,0.00,I9zNlirI,0.740741,0.353357,0.543478,0.549451,0.094098,0.000,0.000000,0.000000,0.0,0.000000,0.000000,0.00,0.000,0.000000,0.000000,0.0,0.000,0.000000,0.000000,0.0,0.000000,0.000000,0.00,0.000,0.000000,0.000000,0.0,0,21,0.00,1.01,0.000,0.000,0.500726,0.007728,NaN,0.00,0.000,inf,0.000000,0.0,0.000000,0.00,0.000,inf,0.000000,0.0,0.000000
394,2024-08-24,Vietnã Vba,09:30,Nhatrang Dolphins,Saigon Heat,3.24,1.28,155.5,1.83,1.83,0.0,0.00,0.00,nFR3qjNP,0.308642,0.781250,0.546448,0.546448,0.089892,154.276,44.090186,0.285788,1.2,1.643168,1.369306,126.96,1.974,0.674411,0.341647,1.0,116.264,20.550530,0.176757,1.8,1.643168,0.912871,127.14,1.452,0.189130,0.130255,30.0,92,78,1.38,1.63,0.000,0.000,0.613243,0.000000,NaN,0.00,0.000,inf,0.000000,0.0,0.000000,0.00,0.000,inf,0.000000,0.0,0.000000
395,2024-08-24,México Lnbp,22:00,Correcaminos,Abejas,1.92,1.80,173.5,1.85,1.85,-1.5,2.05,1.92,0jNEhXg3,0.520833,0.555556,0.540541,0.540541,0.076389,268.924,97.018746,0.360766,1.2,1.643168,1.369306,127.20,3.046,0.919853,0.301987,-19.0,171.018,33.128242,0.193712,0.6,1.341641,2.236068,187.05,2.062,0.421153,0.204245,-40.0,80,87,1.59,2.15,0.000,172.620,0.045620,0.000000,0.046309,0.00,0.000,inf,0.000000,0.0,0.000000,-1.12,-0.224,-3.571429,0.000000,0.0,0.000000
396,2024-08-24,México Lnbp,23:00,Freseros,Soles,2.10,1.68,168.5,1.81,1.89,-3.5,1.9

## Apostas do dia

### Back Home V3 (QCut)

In [9]:
# Duplicando o dataset
df_bh_v3 = jogos.copy()

df_bh_v3 = df_bh_v3[(df_bh_v3['P(H)'] > 0.35) & (df_bh_v3['P(H)'] < 0.65)]
df_bh_v3 = df_bh_v3[(df_bh_v3['CG_H_Last_Game'] > 0) & (df_bh_v3['CG_H_Last_Game'] <= 170)]
df_bh_v3 = df_bh_v3[(df_bh_v3['CG_A_Last_Game'] > 160)]

df_bh_v3.replace([np.inf, -np.inf], 0, inplace=True)

if not df_bh_v3.empty:
    df_bh_v3.to_csv(f'apostas_do_dia/{data_var}/back_home_v3_{data_var}.csv', index=False)

    print('------------ MÉTODO BACK HOME V3 (QCUT) ------------')
    print()

    df_bh_v3['Bet'] = 'Back Home'
    ligs = df_bh_v3['League'].unique().tolist()

    display(df_bh_v3[['Time', 'League', 'Home', 'Away', 'Odds_H', 'Bet']])
    print()
else: print('Sem jogos')

------------ MÉTODO BACK HOME V3 (QCUT) ------------



,Time,League,Home,Away,Odds_H,Bet
395,22:00,México Lnbp,Correcaminos,Abejas,1.92,Back Home
